In [1]:
import os
import gc
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'
print('start')
train = pd.read_feather(PATH + 'train_cleaned.ftr')
print('train')
test = pd.read_feather(PATH + 'test_cleaned.ftr')
print('test')
# PATH = '/home/kai/data/kaggle/talkingdata/data/'
# train = pd.read_csv(PATH + 'train_cleaned_final.csv', nrows=10)
# test = pd.read_csv(PATH + 'test_supplement_cleaned_final.csv', nrows=30)
df = pd.concat([train, test])
train_len = train.shape[0]
cols = ['ip','device','app','os','channel']
fdir = '/home/kai/data/kaggle/talkingdata/haoyandata_1st/'

print(df.head(1))

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

start
train
test
   app  channel  click_id                click_time  click_timestamp  day  \
0    3      379         0 2017-11-06 22:32:21+08:00       1509978741    6   

   dayhourminute  dayhourminute10  device  hour  hourminute  hourminute10  \
0           9992             9990       1    22        1352          1350   

   index     ip  is_attributed  is_test  machine  minute  minute10  os  
0      0  83230              0        0     1013      32        30  13  


In [2]:
def save(df, col_name, train_len):
    df.reset_index(drop=True)
    df[ : train_len].to_feather(fdir + 'train__' + col_name + '.ftr')
    df[train_len : ].reset_index(drop=True).to_feather(fdir + 'test_supplement__' + col_name + '.ftr')

# nmf lsa lda

In [ ]:
from tqdm import tqdm

def MatrixFM(df, key_col_series, value_col_series, n_components, n_jobs = 16):
    
    # first construct sentence and tranlate using countvectorizer
    
    dictionary = {}
    for i in tqdm(range(key_col_series.shape[0])):
        dictionary.setdefault(key_col_series.iloc[i], []).append(str(value_col_series.iloc[i]))

    key_list = list(dictionary.keys()) 
    pos_dict = {key_list[i]:i for i in range(len(key_list))}
    sentences = [' '.join(dictionary[key]) for key in key_list]
    # now get CountVectorize Matrix
    
    cvt = CountVectorizer(token_pattern='\\b\\w+\\b')
    matrix = cvt.fit_transform(sentences)
    
    # LDA part
    
    lda = LatentDirichletAllocation(n_components=n_components, n_jobs=n_jobs)
    lda_matrix = lda.fit_transform(matrix)
    
    # LSA part
    
    lsa = TruncatedSVD(n_components=n_components)
    lsa_matrix = lsa.fit_transform(matrix)
    
    # NMF part
    
    nmf = NMF(n_components=n_components)
    nmf_matrix = nmf.fit_transform(matrix)
    
    # now store all matrix into feather
    
    col = []
    
    for i in ['LDA', 'LSA', 'NMF']:
        for j in range(n_components):
            col.append(i + str(j + 1))
            
    result = np.concatenate((lda_matrix, lsa_matrix, nmf_matrix), axis=1)
    result = pd.DataFrame(result, columns=col)
    
    r = df[x].apply(lambda z: result.iloc[pos_dict[z]])
    return r
    
for x in cols:
    for y in cols:
        if x != y:
            result = MatrixFM(df, df[x], df[y], 5).reset_index(drop=True)
            file_name = 'MatrixFM_' + x + '_' + y
            save(result, file_name, train_len)
            del result
            gc.collect()
            print(x,y)

# click count in next n hours

In [3]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]

for col in feature_col:
    orders[col] = 10 ** (int(np.log(train[col].max() + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

In [ ]:
from collections import defaultdict
from tqdm import tqdm

def _click_count_in_next_n_hour(df, cols, n_hour, time_col):
    # dataframe should maintain a descending order
    rev_train = df.sort_index(ascending=False)
    encodings = get_group(rev_train, cols).values
    times = rev_train[time_col].values
    
    dict_count = defaultdict(int)
    result = []
    bound = 0
    for cur in tqdm(range(len(encodings))):

        while times[bound] - times[cur] > n_hour:
            dict_count[encodings[bound]] -= 1
            bound += 1
        encoding = encodings[cur]
        result.append(dict_count[encoding])
        dict_count[encoding] += 1
    
    r = pd.DataFrame(result[::-1], columns=['click_count_in_next_n_hour_'+'_'.join(cols)+'_'+str(n_hour)])
    return r
        
def _click_count_in_previous_n_hour(df, cols, n_hour, time_col):
    # dataframe should maintain an ascending order
    encodings = get_group(df, cols).values
    times = df[time_col].values
    
    dict_count = defaultdict(int)
    result = []
    bound = 0
    for cur in tqdm(range(len(encodings))):

        while times[cur] - times[bound] > n_hour:
            dict_count[encodings[bound]] -= 1
            bound += 1
        encoding = encodings[cur]
        result.append(dict_count[encoding])
        dict_count[encoding] += 1
    r = pd.DataFrame(result, columns=['click_count_in_previous_n_hour_'+'_'.join(cols)+'_'+str(n_hour)])
    return r

from itertools import combinations
all_cols = []
for i in range(1, len(cols)+1):
    tmp = combinations(cols, i)
    tmp = [list(x) for x in tmp]
    all_cols.extend(tmp)
for x in all_cols:
    tmp = _click_count_in_previous_n_hour(df, x, 1, 'hour')
    file_name = 'click_count_in_previous_n_hour_'+'_'.join(x)+'_1'
    save(tmp, file_name, train_len)
    tmp = _click_count_in_next_n_hour(df, x, 1, 'hour')
    file_name = 'click_count_in_next_n_hour_'+'_'.join(x)+'_1'
    save(tmp, file_name, train_len)
    
    tmp = _click_count_in_previous_n_hour(df, x, 6, 'hour')
    file_name = 'click_count_in_previous_n_hour_'+'_'.join(x)+'_6'
    save(tmp, file_name, train_len)
    tmp = _click_count_in_next_n_hour(df, x, 6, 'hour')
    file_name = 'click_count_in_next_n_hour_'+'_'.join(x)+'_6'
    save(tmp, file_name, train_len)
    del tmp
    gc.collect()
    print(x)

  1%|          | 1682247/242441395 [00:03<07:25, 540800.07it/s]